# **Lab 4:** Advanced `SPARQL`

**Learning Outcomes:**

*   Aggregate results using `GROUP BY`.
*   Perform calculations like `AVG`, `COUNT`, `SUM` on the aggregated data.
*   Sort results using `ORDER BY` to rank your findings.
*   Learning to use nested queries.
*   Create subgraphs with `CONSTRUCT`.
*   Write a federated query using the `SERVICE` keyword to enrich your local data with an external `SPARQL` endpoint.


---


## **Setup:** Install `rdflib`, Import Functions and Populated Graph

In Lab 3, we populated a schema with top 1000 IMDb movies & tv shows. We can import our instances to conduct our advanced queries.

In [1]:
!pip3 install rdflib -U -q

from rdflib import Graph, Literal, Namespace

In [2]:
g = Graph()
g.parse("movie_graph.ttl", format="ttl")

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#gYear, Converter=<function parse_xsd_gyear at 0x000001C33CD4EE50>
Traceback (most recent call last):
  File "c:\Users\Timur\AppData\Local\Programs\Python\Python39\lib\site-packages\rdflib\term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "c:\Users\Timur\AppData\Local\Programs\Python\Python39\lib\site-packages\rdflib\xsd_datetime.py", line 618, in parse_xsd_gyear
    raise ValueError("gYear string must be at least 4 numerals in length")
ValueError: gYear string must be at least 4 numerals in length


<Graph identifier=Nf209e10fd80947df9da2ea25ab838cea (<class 'rdflib.graph.Graph'>)>

Let's run a general query to see if our graph loaded properly before you move onto the advanced `SPARQL` functionalities.

In [3]:
qres = """
SELECT DISTINCT ?s ?p ?o
WHERE {

  ?s ?p ?o .

} LIMIT 10
"""

for row in g.query(qres):
    print(f"{row.s} {row.p} {row.o}")

http://example.org/movies/ToddPhillips http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.org/movies/Director
http://example.org/movies/CowboyBebopTengokunotobira http://example.org/movies/hasActor http://example.org/movies/BeauBillingslea
http://example.org/movies/Brazil http://example.org/movies/hasActor http://example.org/movies/KatherineHelmond
http://example.org/movies/Gladiator http://example.org/movies/hasActor http://example.org/movies/JoaquinPhoenix
http://example.org/movies/Serpico http://example.org/movies/hasGrossRevenue 29800000
http://example.org/movies/TheStraightStory http://example.org/movies/hasDirector http://example.org/movies/DavidLynch
http://example.org/movies/YayanRuhian http://example.org/movies/hasName Yayan Ruhian
http://example.org/movies/TheMachinist http://example.org/movies/releasedInYear 2004
http://example.org/movies/NorikoHidaka http://example.org/movies/hasName Noriko Hidaka
http://example.org/movies/InBruges http://example.org/movies/hasI



---


## **Part 1:** Aggregation with `GROUP BY`
So far, we've only retrieved existing data. But the real power of a query language comes from its ability to calculate new information. To accomplish this, we need to learn how to group our data in a meaningful way. Aggregation is the process of taking many values and summarizing them into groups with the `GROUP BY` functionality. While `GROUP BY` is most often paired with calculation functions like `AVG` or `COUNT`, it can also be used in a powerful way to collect and display all the items associated with a particular group. This is perfect for "discovery" tasks where you want to see the full scope of something in your dataset.

Let's answer the following question by writing a query with `GROUP BY`. We will also use `GROUP_CONCAT` and `AS` functions to format the grouped values to be more human-readable.

In [6]:
q_1 = """
PREFIX movies: <http://example.org/movies/>

SELECT ?directorName ( GROUP_CONCAT(?title; separator=" | ") AS ?filmography ) # formatting
WHERE {
  ?director movies:hasName ?directorName .

  ?movie movies:hasDirector ?director ;
         movies:hasTitle ?title .
}
GROUP BY ?directorName # use GROUP BY outside the WHERE clause
LIMIT 5
"""
for row in g.query(q_1):
  print(f"Director => {row.directorName}")
  print(f"Filmography => {row.filmography}")
  print("-"*20)

Director => Sidney Lumet
Filmography => 12 Angry Men | Dog Day Afternoon | Serpico | The Verdict
--------------------
Director => Steve McQueen
Filmography => 12 Years a Slave
--------------------
Director => Sam Mendes
Filmography => 1917 | American Beauty | Road to Perdition | Skyfall
--------------------
Director => Stanley Kubrick
Filmography => 2001: A Space Odyssey | A Clockwork Orange | Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb | Full Metal Jacket | Spartacus | The Shining
--------------------
Director => Alejandro G. Iñárritu
Filmography => 21 Grams | Amores perros | Birdman or (The Unexpected Virtue of Ignorance) | The Revenant
--------------------




---


## **Part 2:** Computing New Information with `AVG`, `COUNT`, and `SUM`

Now we can make calculations on our groups to reveal new information about our data. Let's start with finding average IMDb rating for each director. Here, we will use `GROUP BY` to group all movies by their director. Then, for each group, we will use the `AVG` function to calculate the average rating. We can also give this new calculated value a name using `AS`.

In [7]:
q_2 = """
PREFIX movies: <http://example.org/movies/>

SELECT ?directorName (AVG(?rating) AS ?avgRating) # use AVG in SELECT clause
WHERE {
    ?director movies:hasName ?directorName .

    ?movie movies:hasDirector ?director ;
           movies:hasImdbRating ?rating .
}
GROUP BY ?directorName
LIMIT 10
"""

print("--- Average Rating per Director ---")
for row in g.query(q_2):
  print(f"{row.directorName}: {round(float(row.avgRating), 2)}")

--- Average Rating per Director ---
Sidney Lumet: 8.1
Steve McQueen: 8.1
Sam Mendes: 8.0
Stanley Kubrick: 8.27
Alejandro G. Iñárritu: 7.85
Spike Lee: 7.72
Danny Boyle: 7.9
Zack Snyder: 7.6
James Mangold: 7.92
Rajkumar Hirani: 8.4


See that we combine our `SPARQL` output with `Python` functions such as `float` and `round` to get more readable printouts.

Let's improve this query by also returning the total number of movies per director in the dataset. We will use the `COUNT` functionality.

In [8]:
q_3 = """
PREFIX movies: <http://example.org/movies/>

SELECT ?directorName (AVG(?rating) AS ?avgRating) (COUNT(?movie) AS ?movieCount)
WHERE {
    ?director movies:hasName ?directorName .

    ?movie movies:hasDirector ?director ;
           movies:hasImdbRating ?rating .
}
GROUP BY ?directorName
LIMIT 10
"""

print("--- Average Rating and Movie Count per Director ---")
for row in g.query(q_3):
  print(f"{row.directorName}: {round(float(row.avgRating), 2)} Rating, {row.movieCount} Movies")

--- Average Rating and Movie Count per Director ---
Sidney Lumet: 8.1 Rating, 4 Movies
Steve McQueen: 8.1 Rating, 1 Movies
Sam Mendes: 8.0 Rating, 4 Movies
Stanley Kubrick: 8.27 Rating, 6 Movies
Alejandro G. Iñárritu: 7.85 Rating, 4 Movies
Spike Lee: 7.72 Rating, 4 Movies
Danny Boyle: 7.9 Rating, 3 Movies
Zack Snyder: 7.6 Rating, 2 Movies
James Mangold: 7.92 Rating, 4 Movies
Rajkumar Hirani: 8.4 Rating, 1 Movies




---


## **Exercise 1:** Further Improving the Query with `SUM`

Calculate the total revenue each director made by using `SUM`.

1.   Modify the `WHERE` clause to reach the gross revenue.
2.   Add the `SUM` (with `AS`) to the `SELECT` clause.
3.   Print a readable director summary.

In [12]:
q_4 = """
PREFIX movies: <http://example.org/movies/>

SELECT ?directorName (AVG(?rating) AS ?avgRating) (COUNT(?movie) AS ?movieCount) (SUM(?revenue) AS ?total_revenue)
WHERE {
    ?director movies:hasName ?directorName .

    ?movie movies:hasDirector ?director ;
          movies:hasGrossRevenue ?revenue;
           movies:hasImdbRating ?rating .
}
GROUP BY ?directorName
LIMIT 10
"""

print("--- Director Summary ---")
for row in g.query(q_4):
  print(f"{row.directorName}: {round(float(row.avgRating), 2)} Rating, {row.movieCount} Movies, Total Revenue {row['total_revenue']}")

--- Director Summary ---
Sidney Lumet: 8.1 Rating, 4 Movies, Total Revenue 138160000
Steve McQueen: 8.1 Rating, 1 Movies, Total Revenue 56671993
Sam Mendes: 8.0 Rating, 4 Movies, Total Revenue 698139284
Stanley Kubrick: 8.27 Rating, 6 Movies, Total Revenue 183813669
Alejandro G. Iñárritu: 7.85 Rating, 4 Movies, Total Revenue 247652802
Spike Lee: 7.72 Rating, 4 Movies, Total Revenue 177289691
Danny Boyle: 7.9 Rating, 3 Movies, Total Revenue 202886628
Zack Snyder: 7.6 Rating, 2 Movies, Total Revenue 318124738
James Mangold: 7.92 Rating, 4 Movies, Total Revenue 517027414
Rajkumar Hirani: 8.4 Rating, 1 Movies, Total Revenue 6532908




---


## **Part 3:** Sorting Results with `ORDER BY`, `DESC`, and `ASC`

Calculating summaries is useful, but it's even more powerful when we can rank the results. The `ORDER BY` clause lets us sort our results. We can sort in ascending order (`ASC`) or descending order (`DESC`). This allows us to create ranked lists and find the "top" or "bottom" items based on some criteria.

Let's start with finding bottom 5 directors by average rating:

In [13]:
q_5 = """
PREFIX movies: <http://example.org/movies/>

SELECT ?directorName (AVG(?rating) AS ?avgRating)
WHERE {
    ?director movies:hasName ?directorName .

    ?movie movies:hasDirector ?director ;
           movies:hasImdbRating ?rating .
}
GROUP BY ?directorName
ORDER BY ASC(?avgRating) # correct usage of ORDER BY
LIMIT 5
"""

print("--- Bottom 5 Directors by Average Rating ---")
for row in g.query(q_5):
    print(f"{row.directorName}: {round(float(row.avgRating), 2)}")

--- Bottom 5 Directors by Average Rating ---
Zack Snyder: 7.6
Jonathan Levine: 7.6
Richard Lester: 7.6
Bradley Cooper: 7.6
Mary Harron: 7.6


Similarly, we can see which actors appear the most in our dataset:

In [14]:
q_6 = """
PREFIX movies: <http://example.org/movies/>

SELECT ?actorName (COUNT(?movie) AS ?movieCount)
WHERE {

    ?actor movies:hasName ?actorName .
    ?movie movies:hasActor ?actor .

}
GROUP BY ?actorName
ORDER BY DESC(?movieCount) # same but descending
LIMIT 10
"""

print("--- Top 10 Most Prolific Actors ---")
for row in g.query(q_6):
    print(f"{row.actorName}: {row.movieCount} movies")

--- Top 10 Most Prolific Actors ---
Robert De Niro: 16 movies
Tom Hanks: 14 movies
Al Pacino: 13 movies
Brad Pitt: 12 movies
Christian Bale: 11 movies
Leonardo DiCaprio: 11 movies
Matt Damon: 11 movies
Clint Eastwood: 11 movies
Denzel Washington: 9 movies
Ethan Hawke: 9 movies




---


## **Part 4:** Subqueries

Sometimes, a query is too complex to solve in a single step. Nested queries, also known as subqueries, allow you to perform a query within your main query. The inner query runs first, and its results can be used, filtered, and joined by the outer query. This is an extremely powerful technique for breaking down complex problems.

A subquery is just a standard `SELECT` query enclosed in curly braces { } within the `WHERE` clause of another query.



```sparql
SELECT ?variable1 ?variable2
WHERE { # outer query
  ?item :someProperty ?variable1 .

  # Inner (Nested) Query
  { SELECT ?variable2 (AGG_FUNCTION(?x) AS ?calculatedValue)
    WHERE {
      # Inner query patterns
    } GROUP BY ?variable2
  }

  # The outer query can now use the results from the inner query
  FILTER(?variable1 > ?calculatedValue)
}
```

Let's see an example query for our graph:

In [15]:
q_7 = """
PREFIX movies: <http://example.org/movies/>

SELECT ?title ?rating
WHERE {
  ?movie movies:hasTitle ?title ;
         movies:hasImdbRating ?rating .

  # Subquery here
  { SELECT (AVG(?r) AS ?overallAverageRating)
    WHERE {
      ?m movies:hasImdbRating ?r .
    }
  }

  FILTER( ?rating > ?overallAverageRating )
} LIMIT 10
"""

print("--- Above Average Movies ---")
for row in g.query(q_7):
    print(f"Movie: {row.title}, Rating: {row.rating}")

--- Above Average Movies ---
Movie: 12 Angry Men, Rating: 9.0
Movie: The Dark Knight, Rating: 9.0
Movie: The Godfather: Part II, Rating: 9.0
Movie: 12 Years a Slave, Rating: 8.1
Movie: Ace in the Hole, Rating: 8.1
Movie: Ah-ga-ssi, Rating: 8.1
Movie: Amores perros, Rating: 8.1
Movie: Before Sunrise, Rating: 8.1
Movie: Before Sunset, Rating: 8.1
Movie: Ben-Hur, Rating: 8.1




---


## **Exercise 2:** Finding Similar Entities with Nested Queries
This is it! All we have learned so far cultivates into this complex query.We will find actors who have similar accomplishments to "Tom Hanks". Our similarity based on career statistics: actors whose average movie rating is within 0.2 points of Tom Hanks's average, and who have a movie count within 1 movie of his total.

Follow the steps below to complete the query and run it:

1.   Calculate average rating and movie counts for all actors in the first subquery.
2.   Get the same stats for Tom Hanks in the second subquery.
3.   Apply conditional filtering based on our similarity conditions.

In [21]:
q_8 = """
PREFIX movies: <http://example.org/movies/>

SELECT ?actorName ?avgRating ?movieCount
WHERE {
  # Subquery 1: Get rating and movie count for all actors
  {
    SELECT ?actorName (AVG(?rt) AS ?avgRating) (COUNT(?movieTitle) AS ?movieCount)
    WHERE {
      ?actor movies:hasName ?actorName .
      ?movie movies:hasActor ?actor ;
             movies:hasImdbRating ?rt ;
             movies:hasTitle ?movieTitle .
    }
    GROUP BY ?actorName
  }

  # Subquery 2: Get stats for Tom Hanks
  {
    SELECT (AVG(?rt_th) AS ?avgRatingTH) (COUNT(?movieTitleTH) AS ?movieCountTH)
    WHERE {
      ?actorTH movies:hasName "Tom Hanks" .
      ?movieTH movies:hasActor ?actorTH ;
               movies:hasImdbRating ?rt_th ;
               movies:hasTitle ?movieTitleTH .
    }
  }

  # Apply filtering based on similarity
  FILTER (
    ABS(?avgRating - ?avgRatingTH) <= 0.2 &&
    ABS(?movieCount - ?movieCountTH) <= 1
  )
}
ORDER BY DESC(?avgRating)
"""


print("--- Similar Careers to Tom Hanks ---")
for row in g.query(q_8):
    print(f"Actor: {row.actorName} | Avg Rating: {round(float(row.avgRating), 2)} | Movie Count: {row.movieCount}")


--- Similar Careers to Tom Hanks ---
Actor: Al Pacino | Avg Rating: 8.08 | Movie Count: 13
Actor: Tom Hanks | Avg Rating: 8.04 | Movie Count: 14


This exercise shows how powerful `SPARQL` queries can be. Also, notice that the query took a while to compute since the complexity is higher than any example we have seen before.


---


## [OPTIONAL] **Part 5:** Extracting Subgraphs with `CONSTRUCT`

So far, all our queries have used `SELECT`, which returns a table of results. `SPARQL` has other query forms, and one of the most powerful is `CONSTRUCT`. Instead of a table, a `CONSTRUCT` query returns a new RDF graph. You can simplify your data by excluding certian elements, or create a subgraph including a specific subset of your data.

Let's create a subgraph, containing only Sci-Fi genre movies, but only including their director and title information:

In [22]:
q_9 = """
PREFIX movies: <http://example.org/movies/>

CONSTRUCT { # template for our new graph
    ?movieURI movies:hasTitle ?title ;
              movies:hasGenre ?genreURI ;
              movies:hasDirector ?directorURI .

    ?directorURI movies:hasName ?directorName .
}
WHERE { # this is where we filter for Sci-Fi movies

    ?movieURI movies:hasGenre "Sci-Fi" ;
              movies:hasTitle ?title ;
              movies:hasDirector ?directorURI .

    ?directorURI movies:hasName ?directorName .
}
"""

# The result of a CONSTRUCT query is a new rdflib Graph object
scifi_subgraph_result = g.query(q_9)
scifi_graph = scifi_subgraph_result.graph

# We can check its type and size
print(f"The result is a: {type(scifi_graph)}")
print(f"The new subgraph has {len(scifi_graph)} triples.")

# Let's print the new graph in Turtle format to see the result
print("\n--- Sci-Fi Movies Subgraph ---")
print(scifi_graph.serialize(format="turtle"))

The result is a: <class 'rdflib.graph.Graph'>
The new subgraph has 150 triples.

--- Sci-Fi Movies Subgraph ---
@prefix ns1: <http://example.org/movies/> .

ns1:2001ASpaceOdyssey ns1:hasDirector ns1:StanleyKubrick ;
    ns1:hasTitle "2001: A Space Odyssey" .

ns1:28DaysLater ns1:hasDirector ns1:DannyBoyle ;
    ns1:hasTitle "28 Days Later..." .

ns1:AClockworkOrange ns1:hasDirector ns1:StanleyKubrick ;
    ns1:hasTitle "A Clockwork Orange" .

ns1:Alien ns1:hasDirector ns1:RidleyScott ;
    ns1:hasTitle "Alien" .

ns1:Aliens ns1:hasDirector ns1:JamesCameron ;
    ns1:hasTitle "Aliens" .

ns1:Arrival ns1:hasDirector ns1:DenisVilleneuve ;
    ns1:hasTitle "Arrival" .

ns1:AvengersInfinityWar ns1:hasDirector ns1:AnthonyRusso ;
    ns1:hasTitle "Avengers: Infinity War" .

ns1:BacktotheFuture ns1:hasDirector ns1:RobertZemeckis ;
    ns1:hasTitle "Back to the Future" .

ns1:BacktotheFuturePartII ns1:hasDirector ns1:RobertZemeckis ;
    ns1:hasTitle "Back to the Future Part II" .

ns1:BladeRun



---


## [OPTIONAL] **Part 6:** Federated Queries with `SERVICE` or `SPARQLWrapper`

Our graph is powerful, but instances are isolated within the dataset. The true power of the Semantic Web comes from connecting our data to higher level concepts in external schemas. A federated query is a query that retrieves data from more than one knowledge graph at the same time.

In `SPARQL`, we do this with the `SERVICE` keyword. You provide the URL of an external `SPARQL` endpoint, and the query inside the `SERVICE` block will be executed on that remote server.

By using this, we will be able to enrich our local data. We can find an entity in a graoh and use a massive public knowledge base like Wikidata to pull in additional facts that we don't have.

Here is an example, querying Wikidata to retrieve facts about the director "Christopher Nolan":


```sparql
PREFIX movies: <http://example.org/movies/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wdt: <http://www.wikidata.org/prop/direct/> # Wikidata namespace

SELECT ?directorName ?dateOfBirth
WHERE {

    ?director movies:hasName ?directorName .
    FILTER(CONTAINS(?directorName, "Christopher Nolan"))

    # Query the Wikidata endpoint
    SERVICE <https://query.wikidata.org/sparql> {
        ?person rdfs:label ?directorName .

        # Get their date of birth (property P569 in Wikidata).
        ?person wdt:P569 ?dateOfBirth .

        FILTER( LANG(?directorName) = "en" )
    }
}
LIMIT 1
```
This query will not return anything since `rdflib` is not able to request information from external sites. There are other tools such as Virtuoso or Apache Jena Fuseki that would support the `SERVICE` functionality. Instead we can retrieve information from our local graph, then use the `SPARQLWrapper` library to write external queries. Let's find the birtdate of first 25 directors:

In [23]:
# Install the library and import useful functions
!pip3 install SPARQLWrapper -U -q
from SPARQLWrapper import SPARQLWrapper, JSON

# Setup connection to the Wikidata
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

# Get the first 25 director names
q_10 = """
PREFIX movies: <http://example.org/movies/>

SELECT DISTINCT ?directorName
WHERE {
  ?d movies:hasName ?directorName .
} LIMIT 25
"""
local_results = g.query(q_10)


# Iterate through names and get birth dates for each one
for row in local_results:
  director_name = str(row.directorName)

  # Note: We constrain the search to humans (wd:Q5) to get more accurate results.
  remote_query = f"""
  PREFIX wdt: <http://www.wikidata.org/prop/direct/>
  PREFIX wd: <http://www.wikidata.org/entity/>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

  SELECT ?dateOfBirth
  WHERE {{
    ?person wdt:P31 wd:Q5 ; # assure that the entity is a human
            rdfs:label "{director_name}"@en ;
            wdt:P569 ?dateOfBirth .
  }} LIMIT 1
  """
  sparql.setQuery(remote_query)

  try:
    # execute
    results = sparql.query().convert()
    bindings = results["results"]["bindings"]

    if bindings:
      dob = bindings[0]["dateOfBirth"]["value"]
      print(f"Director: {director_name}, DOB: {dob}")

    else: # in case director is not on Wikidata
      print(f"Director: {director_name}, DOB: {None}")

  except Exception as e: # catch any errors
    print(f"An error occurred for {director_name}: {e}")

Director: Aaron Eckhart, DOB: 1968-03-12T00:00:00Z
Director: Aaron Taylor-Johnson, DOB: 1990-06-13T00:00:00Z
Director: Abdellatif Kechiche, DOB: 1960-12-07T00:00:00Z
Director: Abhishek Kapoor, DOB: 1971-08-06T00:00:00Z
Director: Abigail Breslin, DOB: 1996-04-14T00:00:00Z
Director: Adam Baldwin, DOB: 1962-02-27T00:00:00Z
Director: Adam Driver, DOB: 1983-11-19T00:00:00Z
Director: Adam McKay, DOB: 1968-04-17T00:00:00Z
Director: Adel Bencherif, DOB: 1975-05-30T00:00:00Z
An error occurred for Adil Hussain: HTTP Error 429: Too Many Requests
An error occurred for Adrian Molina: HTTP Error 429: Too Many Requests
Director: Adèle Exarchopoulos, DOB: 1993-11-22T00:00:00Z
Director: Adèle Haenel, DOB: 1989-02-11T00:00:00Z
Director: Agnes Moorehead, DOB: 1900-12-06T00:00:00Z
Director: Ahna Capri, DOB: 1944-07-06T00:00:00Z
Director: Ahney Her, DOB: 1992-07-13T00:00:00Z
Director: Aidan Gillen, DOB: 1968-04-24T00:00:00Z
Director: Aitana Sánchez-Gijón, DOB: 1968-11-05T00:00:00Z
Director: Ajay Naidu, DOB

You can also use query Wikidata online by visiting this URL: https://query.wikidata.org/

<ins>**Note:**</ins> Observe that we used the `DISTINCT` functionality in our local query to return unique insrances, which may occur on less structured graphs. Our graph contains a unique URI for each director, so duplicates are avoided altogether.

